BigMac Index

Dataset: BigMacPrice --> https://www.kaggle.com/datasets/vittoriogiatti/bigmacprice?resource=download

PPI: https://data.oecd.org/price/producer-price-indices-ppi.htm#indicator-chart

Forex: https://pypi.org/project/forex-python/

Questions:
- Can we predict (some price indicator)
- How did the price develop
- Does it grow at the same pace
- Does the exchange rate reflect the price difference

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import pycountry
import requests
import json

from datetime import datetime, timedelta

%matplotlib inline

df_bigmac = pd.read_csv('BigMacPrice.csv')
df_bigmac.head()

,date,currency_code,name,local_price,dollar_ex,dollar_price
0,2000-04-01,ARS,Argentina,2.50,1,2.50
1,2000-04-01,AUD,Australia,2.59,1,2.59
2,2000-04-01,BRL,Brazil,2.95,1,2.95
3,2000-04-01,GBP,Britain,1.90,1,1.90
4,2000-04-01,CAD,Canada,2.85,1,2.85


In [2]:
df_ppi = pd.read_csv('PPI_23112022084950389.csv')
df_ppi.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUT,PPI,DOMESTIC,AGRWTH,A,2000,3.405336,NaN
1,AUT,PPI,DOMESTIC,AGRWTH,A,2001,0.048577,NaN
2,AUT,PPI,DOMESTIC,AGRWTH,A,2002,-1.408040,NaN
3,AUT,PPI,DOMESTIC,AGRWTH,A,2003,0.285630,NaN
4,AUT,PPI,DOMESTIC,AGRWTH,A,2004,2.219603,NaN


In [1]:
def map_country(country_str):
        try:
            return pycountry.countries.lookup(country_str).name
        except LookupError:
            return None
        except Exception as e:
            print(e)
            raise ValueError


def convert_exchange_rate(base, out_curr, date):
    try:
        # api url for request
        date_plus1 = datetime.strptime(date, '%Y-%m-%d')
        # Prices are monthly, so get the exchange rate at approx. half of the month
        date_plus1 = datetime.strftime(date_plus1 + timedelta(days=15), '%Y-%m-%d')
        url = 'https://api.exchangerate.host/timeseries?base={0}&start_date={1}&end_date={2}&symbols={3}'.format(base,
                                                                                                                 date_plus1,
                                                                                                                 date_plus1,
                                                                                                                 out_curr)
        response = requests.get(url)
        # retrive response in json format
        data = response.json()

        return data['rates'][date_plus1][out_curr]
    except Exception as e:
        return None

            
def clean_ppi(df_ppi, measure):
    # Filter PPI
    f1 = df_ppi.TIME.str.contains("-")
    f2 = df_ppi.TIME.str.contains('Q')
    f3 = df_ppi.SUBJECT == "DOMESTIC"
    f4 = df_ppi.MEASURE == measure
    mask = f1 & ~f2 &  f3 & f4

    df_ppi = df_ppi[mask]

    df_ppi['TIME'] = pd.to_datetime(df_ppi['TIME'])

    df_ppi['country'] = df_ppi['LOCATION'].apply(map_country)
    df_ppi = df_ppi[~df_ppi['country'].isnull()]
    return df_ppi


def clean_df_bigmac(df_bigmac):
    
    # Use same mapping as for df_ppi
    df_bigmac['country'] = df_bigmac.name.apply(map_country)
    
    # Few countries unable to map in pycountry - so do manually
    remap = {'Britain': 'United Kingdom',
             'Russia': 'Russian Federation',
             'UAE': 'United Arab Emirates'}
    df_bigmac['country'] = df_bigmac['name'].replace(remap)
    
    # Seems to be an error in consistency of the date column for the given dates
    df_bigmac.loc[df_bigmac.date == "2019-07-09", "date"] = "2019-07-01"
    df_bigmac.loc[df_bigmac.date == "2020-01-14", "date"] = "2020-01-01"
    
    # Map Exchange Rates
    df_bigmac['dollar_ex_adjusted'] = df_bigmac.apply(lambda x: convert_exchange_rate(base="USD",
                                                                                      out_curr=x.currency_code,
                                                                                      date=x.date),
                                                         axis=1)
    
    # Set col date col to datetime for joining purposes
    df_bigmac['date'] = pd.to_datetime(df_bigmac['date'])
    
    return df_bigmac

In [4]:
# Clean Data Frames
df_ppi_cleaned = clean_ppi(df_ppi, "IDX2015")
df_bigmac_cleaned = clean_df_bigmac(df_bigmac=df_bigmac)

C:\Users\C77561\AppData\Local\Temp\ipykernel_20160\1492013466.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ppi['TIME'] = pd.to_datetime(df_ppi['TIME'])
C:\Users\C77561\AppData\Local\Temp\ipykernel_20160\1492013466.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ppi['country'] = df_ppi['LOCATION'].apply(map_country)


In [5]:
# Put together in 1 DataFrame, and put all rows in perspective to the US ($)
df = pd.merge(df_bigmac, df_ppi_cleaned, left_on=['date', 'country'], right_on=['TIME', 'country'], how='left')
df = df.set_index('date')

# Create a seperate dataframe with only USA prices, to join
df_comparison = df[df['country'] == "United States"]
df = df.join(df_comparison[['local_price']], how='left', rsuffix="_comparison")

# Set expected prices and difference based on exchange rates in local currency
df['expected_price'] = round(df['local_price_comparison'] * df['dollar_ex_adjusted'], 2)
df['diff_local'] = round(df['local_price'] - df['expected_price'], 2)

# Set expected prices and difference based on exchange rates in local currency
df['dollar_price_adjusted'] = round(df['local_price'] / df['dollar_ex_adjusted'], 2)
df['diff_dollar'] = round(df['dollar_price_adjusted'] - df['local_price_comparison'], 2)

# Also determing percentual difference
df['perc_diff'] = round((df['diff_dollar'] / df['local_price_comparison'])*100, 2)

df = df.reset_index()

df.head()

,date,currency_code,name,local_price,dollar_ex,dollar_price,country,dollar_ex_adjusted,LOCATION,INDICATOR,...,FREQUENCY,TIME,Value,Flag Codes,local_price_comparison,expected_price,diff_local,dollar_price_adjusted,diff_dollar,perc_diff
0,2000-04-01,ARS,Argentina,2.50,1,2.50,Argentina,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,2.24,NaN,NaN,NaN,NaN,NaN
1,2000-04-01,AUD,Australia,2.59,1,2.59,Australia,1.677044,NaN,NaN,...,NaN,NaT,NaN,NaN,2.24,3.76,-1.17,1.54,-0.70,-31.25
2,2000-04-01,BRL,Brazil,2.95,1,2.95,Brazil,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,2.24,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,GBP,Britain,1.90,1,1.90,United Kingdom,0.630608,GBR,PPI,...,M,2000-04-01,73.8346,NaN,2.24,1.41,0.49,3.01,0.77,34.38
4,2000-04-01,CAD,Canada,2.85,1,2.85,Canada,1.474109,NaN,NaN,...,NaN,NaT,NaN,NaN,2.24,3.30,-0.45,1.93,-0.31,-13.84


In [6]:
# Select only columns of interest
cols = ['date', 'currency_code', 'name', 'local_price', 'dollar_ex_adjusted',
        'Value', 'local_price_comparison', 'expected_price', 'diff_local',
       'dollar_price_adjusted', 'diff_dollar', 'perc_diff']

df = df[cols]

In [7]:
# Pickle Dataset, because the creation takes too long
df.to_pickle('BigMacPrices_cleaned')

In [8]:
df.head()

,date,currency_code,name,local_price,dollar_ex_adjusted,Value,local_price_comparison,expected_price,diff_local,dollar_price_adjusted,diff_dollar,perc_diff
0,2000-04-01,ARS,Argentina,2.50,NaN,NaN,2.24,NaN,NaN,NaN,NaN,NaN
1,2000-04-01,AUD,Australia,2.59,1.677044,NaN,2.24,3.76,-1.17,1.54,-0.70,-31.25
2,2000-04-01,BRL,Brazil,2.95,NaN,NaN,2.24,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,GBP,Britain,1.90,0.630608,73.8346,2.24,1.41,0.49,3.01,0.77,34.38
4,2000-04-01,CAD,Canada,2.85,1.474109,NaN,2.24,3.30,-0.45,1.93,-0.31,-13.84


In [4]:
# Filter PPI
    

IndentationError: unexpected indent (357377627.py, line 2)

In [5]:

df_cpi.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,CPI,ENRG,AGRWTH,A,1972,4.910070,NaN
1,AUS,CPI,ENRG,AGRWTH,A,1973,3.762801,NaN
2,AUS,CPI,ENRG,AGRWTH,A,1974,13.173540,NaN
3,AUS,CPI,ENRG,AGRWTH,A,1975,19.422470,NaN
4,AUS,CPI,ENRG,AGRWTH,A,1976,8.833195,NaN


In [12]:
df_ppi = pd.read_csv('CPI_07122022080621228.csv')

f1 = df_ppi.TIME.str.contains("-")
f2 = df_ppi.TIME.str.contains('Q')
f3 = df_ppi.SUBJECT == "TOT"
f4 = df_ppi.MEASURE == "IDX2015"
mask = f1 & ~f2 & f3 & f4

df_ppi = df_ppi[mask]

df_ppi['TIME'] = pd.to_datetime(df_ppi['TIME'])

df_ppi['country'] = df_ppi['LOCATION'].apply(map_country)
df_ppi = df_ppi[~df_ppi['country'].isnull()]


df_ppi

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,country
189554,AUT,CPI,TOT,IDX2015,M,1958-01-01,15.56513,NaN,Austria
189555,AUT,CPI,TOT,IDX2015,M,1958-02-01,15.59639,NaN,Austria
189556,AUT,CPI,TOT,IDX2015,M,1958-03-01,15.58076,NaN,Austria
189557,AUT,CPI,TOT,IDX2015,M,1958-04-01,15.83080,NaN,Austria
189558,AUT,CPI,TOT,IDX2015,M,1958-05-01,16.03396,NaN,Austria
...,...,...,...,...,...,...,...,...,...
279817,ZAF,CPI,TOT,IDX2015,M,2022-06-01,140.83490,NaN,South Africa
279818,ZAF,CPI,TOT,IDX2015,M,2022-07-01,143.13040,NaN,South Africa
279819,ZAF,CPI,TOT,IDX2015,M,2022-08-01,143.53550,NaN,South Africa
279820,ZAF,CPI,TOT,IDX2015,M,2022-09-01,143.67050,NaN,South Africa


In [27]:
df_ppi = pd.read_csv('CPI_07122022080621228.csv')
df_ppi[(df_ppi.LOCATION == "NLD") &
       (df_ppi.FREQUENCY == "A") &
       (df_ppi.MEASURE == "IDX2015") &
       (df_ppi.TIME > "2013")]

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
147565,NLD,CPI,ENRG,IDX2015,A,2014,105.62500,NaN
147566,NLD,CPI,ENRG,IDX2015,A,2015,99.99834,NaN
147567,NLD,CPI,ENRG,IDX2015,A,2016,94.45167,NaN
147568,NLD,CPI,ENRG,IDX2015,A,2017,97.86584,NaN
147569,NLD,CPI,ENRG,IDX2015,A,2018,105.86670,NaN
147570,NLD,CPI,ENRG,IDX2015,A,2019,114.04330,NaN
147571,NLD,CPI,ENRG,IDX2015,A,2020,103.69830,NaN
147572,NLD,CPI,ENRG,IDX2015,A,2021,121.59580,NaN
147626,NLD,CPI,FOOD,IDX2015,A,2014,99.35667,NaN
147627,NLD,CPI,FOOD,IDX2015,A,2015,100.00080,NaN


In [20]:
df_ppi[(df_ppi.LOCATION == "NLD") & (df_ppi.TIME > "2014-12-01")]

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes,country
234813,NLD,CPI,TOT,IDX2015,M,2015-01-01,98.15,NaN,Netherlands
234814,NLD,CPI,TOT,IDX2015,M,2015-02-01,98.69,NaN,Netherlands
234815,NLD,CPI,TOT,IDX2015,M,2015-03-01,99.75,NaN,Netherlands
234816,NLD,CPI,TOT,IDX2015,M,2015-04-01,100.39,NaN,Netherlands
234817,NLD,CPI,TOT,IDX2015,M,2015-05-01,100.61,NaN,Netherlands
...,...,...,...,...,...,...,...,...,...
234902,NLD,CPI,TOT,IDX2015,M,2022-06-01,119.05,NaN,Netherlands
234903,NLD,CPI,TOT,IDX2015,M,2022-07-01,121.57,NaN,Netherlands
234904,NLD,CPI,TOT,IDX2015,M,2022-08-01,123.95,NaN,Netherlands
234905,NLD,CPI,TOT,IDX2015,M,2022-09-01,126.89,NaN,Netherlands
